In [1]:
import os
import numpy as np
os.chdir(os.path.abspath('../')) # to gpt folder

import torch
import torch.nn as nn

# For training and fine-tuning
We want to see how do losses evolve as more pixels become correct. This is to decide the target losses we want to achieve.

In [2]:
batch_size=16
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
from src.gpt_mnist.sampler import Pytorch_GPT_MNIST_Sampler
samp = Pytorch_GPT_MNIST_Sampler(compenv_mode=None, growth_mode=None)

In [4]:
from src.gpt_mnist.model import ResGPTNet34
net = ResGPTNet34(nG0=samp.gen.nG0, Nj=samp.gen.N_neighbor)
net.to(device=device)
print('')

In [5]:
x, y0, yg0, ys0 = samp.get_sample_batch_uniform_random(batch_size=batch_size, device=device)
print('y0.shape:',y0.shape)
print('yg0.shape:',yg0.shape)
print('ys0.shape:',ys0.shape)

y0.shape: torch.Size([16])
yg0.shape: torch.Size([16, 28, 28])
ys0.shape: torch.Size([16, 28, 28])


In [6]:
y, yg, ys = net(x)
print('y.shape:',y.shape)
print('yg.shape:',yg.shape)
print('ys.shape:',ys.shape)

y.shape: torch.Size([16, 10])
yg.shape: torch.Size([16, 4, 28, 28])
ys.shape: torch.Size([16, 8, 28, 28])


## for y

In [7]:
criterion = nn.CrossEntropyLoss(reduction='mean')
class_pred_loss = criterion(y,y0)
print('before',class_pred_loss)
for i in range(batch_size):
    y[i,y0[i]]+=100
    class_pred_loss = criterion(y,y0)
    print('after %s'%(str(i)),class_pred_loss.item())


before tensor(2.7760, device='cuda:0', grad_fn=<NllLossBackward>)
after 0 2.5924909114837646
after 1 2.510190963745117
after 2 2.220311164855957
after 3 2.0286989212036133
after 4 1.9016348123550415
after 5 1.662554383277893
after 6 1.5309978723526
after 7 1.3936432600021362
after 8 1.250139832496643
after 9 1.0544960498809814
after 10 0.7858767509460449
after 11 0.6200230121612549
after 12 0.5138240456581116
after 13 0.40636447072029114
after 14 0.15562108159065247
after 15 0.0


## for yg

In [8]:
gc_weights = np.ones(shape=(samp.gen.nG0+1))
gc_weights[0]= 1e-2
gc_weights = torch.tensor(gc_weights).to(device=device).to(torch.float)

criterion = nn.CrossEntropyLoss(reduction='mean', weight=gc_weights)
gen_config_loss = criterion(yg, yg0)
print(gen_config_loss)
for i in range(batch_size):
    for j in range(28):
        for k in range(28):
            yg[i,yg0[i,j,k],j,k]+=1000 
            
            midloss = criterion(yg, yg0)
            if i>batch_size-3 or i==0:
                if (k+1)%7==0 and (j+1)%7==0:
                    print('      %3s,%3s'%(str(j),str(k)),midloss.item())
    gen_config_loss = criterion(yg, yg0)
    if i>batch_size-3 or i==0:
        print('after %s'%(str(i)),gen_config_loss.item())

tensor(1.4170, device='cuda:0', grad_fn=<NllLoss2DBackward>)
        6,  6 1.4028180837631226
        6, 13 1.3937290906906128
        6, 20 1.3931777477264404
        6, 27 1.3926008939743042
       13,  6 1.3785380125045776
       13, 13 1.3779854774475098
       13, 20 1.3774220943450928
       13, 27 1.3768510818481445
       20,  6 1.3627784252166748
       20, 13 1.3622063398361206
       20, 20 1.3616459369659424
       20, 27 1.3610666990280151
       27,  6 1.347006916999817
       27, 13 1.3464280366897583
       27, 20 1.3458528518676758
       27, 27 1.3452743291854858
after 0 1.3452742099761963
        6,  6 0.16692522168159485
        6, 13 0.16637453436851501
        6, 20 0.165832057595253
        6, 27 0.16525940597057343
       13,  6 0.1332816183567047
       13, 13 0.13273952901363373
       13, 20 0.13216015696525574
       13, 27 0.1315852701663971
       20,  6 0.10812195390462875
       20, 13 0.1075555831193924
       20, 20 0.10699226707220078
       20, 27 0.

## for ys

In [9]:
gt_weights = np.ones(shape=(samp.gen.N_neighbor))
gt_weights[0]= 1e-2
gt_weights = torch.tensor(gt_weights).to(device=device).to(torch.float)
criterion = nn.CrossEntropyLoss(reduction='mean', weight=gt_weights)

gen_transform_loss = criterion(ys, ys0)
print(gen_transform_loss)
for i in range(batch_size):
    for j in range(28):
        for k in range(28):
            ys[i,ys0[i,j,k],j,k]+=1000 
            midloss = criterion(ys, ys0)
            if i>batch_size-3 or i==0:
                if (k+1)%7==0 and (j+1)%7==0:
                    print('      %3s,%3s'%(str(j),str(k)),midloss.item())
    gen_transform_loss = criterion(ys, ys0)
    if i>batch_size-3 or i==0:
        print('after %s'%(str(i)),gen_transform_loss.item())

tensor(2.1026, device='cuda:0', grad_fn=<NllLoss2DBackward>)
        6,  6 2.0800485610961914
        6, 13 2.0662407875061035
        6, 20 2.0653350353240967
        6, 27 2.06442928314209
       13,  6 2.0417771339416504
       13, 13 2.040858030319214
       13, 20 2.0399444103240967
       13, 27 2.0390334129333496
       20,  6 2.0163612365722656
       20, 13 2.015460252761841
       20, 20 2.014561891555786
       20, 27 2.0136618614196777
       27,  6 1.9910122156143188
       27, 13 1.9901154041290283
       27, 20 1.989219307899475
       27, 27 1.9883235692977905
after 0 1.9883235692977905
        6,  6 0.24443276226520538
        6, 13 0.24352847039699554
        6, 20 0.24263063073158264
        6, 27 0.24172550439834595
       13,  6 0.19366104900836945
       13, 13 0.1927385777235031
       13, 20 0.19181208312511444
       13, 27 0.19088876247406006
       20,  6 0.1682184934616089
       20, 13 0.16732251644134521
       20, 20 0.16642244160175323
       20, 27 0.16